In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset


import torchvision.transforms as transforms
import torchvision

from PIL import Image

from network.feature_extractor import ResNet
from network.feature_extractor import ResnetOriginal
from network.training import PretrainTraining
from network.dataset_chexpert import ChestXrayDataset

random_seed = 42
BATCH_SIZE = 4

# Get data (chexpert)

In [ ]:
path_folder = "/Users/carlasagebiel/Documents/AML_project/data/"
weight_path = "/Users/carlasagebiel/Documents/AML_project/_weights"

In [ ]:
train_path = "/Users/carlasagebiel/Documents/AML_project/_data/CheXpert-v1.0-small/train"
valid_path = "/Users/carlasagebiel/Documents/AML_project/_data/CheXpert-v1.0-small/valid"

In [ ]:
train_data = pd.read_csv("/Users/carlasagebiel/Documents/AML_project/_data/CheXpert-v1.0-small/train.csv")
valid_data = pd.read_csv("/Users/carlasagebiel/Documents/AML_project/_data/CheXpert-v1.0-small/valid.csv")

In [ ]:
LABELS = train_data.columns
LABELS 

In [ ]:
train, test = train_test_split(train_data, test_size= 0.1, random_state = random_seed)

In [ ]:
labels = ['Path', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax','Support Devices']
train_dataset = ChestXrayDataset(path_folder,train_data, labels)
val_dataset = ChestXrayDataset(path_folder, valid_data, labels)

In [ ]:
weights = train_dataset.get_loss_weights()

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size= BATCH_SIZE, shuffle = True)

# Get model

In [ ]:
model = ResnetOriginal(type = "resnet18", shapes = [512, 128, 314], trainable_resnet = True, 
                       trainable_level=  4, sigmoid_activation=False)

In [ ]:
loss = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(weights))

In [ ]:
training = PretrainTraining("resnet18_example", model, loss,
                            batch_size= BATCH_SIZE, verbose_level= 2, path_dir = weight_path,
                            data_trafo= False)

In [ ]:
training.train(3, train_dataloader)